# Notebook to create `expressions` module

In [1]:
from pathlib import Path

PACKAGE = "mintalib"
ROOTDIR = Path.cwd().parent
PKGDIR = ROOTDIR.joinpath(f"src/{PACKAGE}").resolve(strict=True)

if '__file__' in globals():
    print(f"Running {__file__} ...")


In [2]:
import polars as pl
from mintalib.samples import sample_prices

prices = sample_prices()
prices = pl.from_pandas(prices, include_index=True, nan_to_null=True)
prices


date,open,high,low,close,volume
datetime[ns],f64,f64,f64,f64,i64
1980-12-12 00:00:00,0.098943,0.099373,0.098943,0.098943,469033600
1980-12-15 00:00:00,0.094211,0.094211,0.093781,0.093781,175884800
1980-12-16 00:00:00,0.087328,0.087328,0.086898,0.086898,105728000
1980-12-17 00:00:00,0.089049,0.089479,0.089049,0.089049,86441600
1980-12-18 00:00:00,0.09163,0.092061,0.09163,0.09163,73449600
…,…,…,…,…,…
2024-10-15 00:00:00,233.610001,237.490005,232.369995,233.850006,64751400
2024-10-16 00:00:00,231.600006,232.119995,229.839996,231.779999,34082200
2024-10-17 00:00:00,233.429993,233.850006,230.520004,232.149994,32993800


In [3]:
PRELUDE='''
"""Expressions Module"""

# Do not Edit! This file was generated.

import warnings
from mintalib import core
from mintalib.model.expression import wrap_expression

warnings.warn(
    f"Module {__name__} is deprecated and will be removed in a future release!", 
    DeprecationWarning,
    stacklevel=2)


'''

exec(PRELUDE)

/var/folders/s8/jtb72jcx6739t9rmc8pl26980000gn/T/ipykernel_16241/1150788836.py:18: DeprecationWarning: Module __main__ is deprecated and will be removed in a future release!
  exec(PRELUDE)


In [4]:
@wrap_expression(core.calc_sma)
def SMA(src, *, period: int): ...


prices.select(
    SMA('close', period=5).alias('s1'),
    SMA(pl.col('close'), period=5).alias('s2'),
    pl.col('close').pipe(SMA, period=5).alias('s3')
)


s1,s2,s3
f64,f64,f64
null,null,null
null,null,null
null,null,null
null,null,null
0.09206,0.09206,0.09206
…,…,…
230.256,230.256,230.256
230.704001,230.704001,230.704001
231.326001,231.326001,231.326001


In [5]:
@wrap_expression(core.calc_macd)
def MACD(src): ...

prices.select(
    MACD('close').struct.unnest()
)


macd,macdsignal,macdhist
f64,f64,f64
null,null,null
null,null,null
null,null,null
null,null,null
null,null,null
…,…,…
1.815958,1.313965,0.501993
1.941114,1.439395,0.501719
2.046565,1.560829,0.485736


In [6]:
@wrap_expression(core.calc_atr)
def ATR(period: int, *, src=None): ...

prices.select(ATR(14))

atr
f64
null
null
null
null
null
…
4.516121
4.479971
4.39783


In [7]:
import inspect

from mintalib.builder import annotate_parameter

def make_signature(calc_func):
    """creates function signature from core function"""
    sig = inspect.signature(calc_func)
    first_param = next(iter(sig.parameters.values()))
    
    arg_params = []
    for param in sig.parameters.values():
        if param.name in ("wrap", "prices"):
            continue

        if param.name == "series":
            param = inspect.Parameter(
                name="src",
                kind=inspect.Parameter.POSITIONAL_OR_KEYWORD
            )

        param = annotate_parameter(param)
        arg_params.append(param)

    if first_param.name == "prices":
        param = inspect.Parameter(
            name="src",
            default='*',
            kind=inspect.Parameter.KEYWORD_ONLY
        )
        arg_params.append(param)

    return sig.replace(parameters=arg_params)

sig = make_signature(core.calc_sma)
print(sig)



(src, period: int)


In [8]:

def make_expression(calc_func):
    cname = f"core.{calc_func.__name__}"
    fname = calc_func.__name__.removeprefix("calc_").upper()
    signature = make_signature(calc_func)

    buffer = f"@wrap_expression({cname})\n"
    buffer += f"def {fname}{signature}: ...\n"
    return buffer


for cf in core.calc_sma, core.calc_macd, core.calc_atr:  
    code = make_expression(cf)
    print(code)



@wrap_expression(core.calc_sma)
def SMA(src, period: int): ...

@wrap_expression(core.calc_macd)
def MACD(src, n1: int = 12, n2: int = 26, n3: int = 9): ...

@wrap_expression(core.calc_atr)
def ATR(period: int = 14, *, src='*'): ...



In [ ]:
def core_functions(exclude = ("calc_eval",)):
    names = sorted(k for k, v in vars(core).items() if k.startswith("calc_") and callable(v))
    if exclude:
        names = [n for n in names if n not in exclude]
    return names

core_functions()

['calc_abs',
 'calc_adx',
 'calc_alma',
 'calc_atr',
 'calc_avgprice',
 'calc_bbands',
 'calc_bbp',
 'calc_bbw',
 'calc_bop',
 'calc_cci',
 'calc_clag',
 'calc_cmf',
 'calc_crossover',
 'calc_crossunder',
 'calc_curve',
 'calc_dema',
 'calc_diff',
 'calc_dmi',
 'calc_ema',
 'calc_exp',
 'calc_flag',
 'calc_hma',
 'calc_kama',
 'calc_keltner',
 'calc_ker',
 'calc_lag',
 'calc_log',
 'calc_lroc',
 'calc_macd',
 'calc_macdv',
 'calc_mad',
 'calc_mav',
 'calc_max',
 'calc_mdi',
 'calc_mfi',
 'calc_midprice',
 'calc_min',
 'calc_natr',
 'calc_pdi',
 'calc_ppo',
 'calc_price',
 'calc_qsf',
 'calc_rma',
 'calc_roc',
 'calc_rsi',
 'calc_rvalue',
 'calc_sar',
 'calc_shift',
 'calc_sign',
 'calc_slope',
 'calc_sma',
 'calc_stdev',
 'calc_step',
 'calc_stoch',
 'calc_streak',
 'calc_sum',
 'calc_tema',
 'calc_trange',
 'calc_tsf',
 'calc_typprice',
 'calc_updown',
 'calc_wclprice',
 'calc_wma']

In [10]:
from pprint import pformat

import importlib.util

def make_expressions(cnames=None):
    if cnames is None:
        cnames = core_functions()

    output = PRELUDE

    fnames = []

    for cname in cnames:
        func = getattr(core, cname)
        name = cname.removeprefix("calc_").upper()
        code = make_expression(func)
        fnames.append(name)
        output += code + "\n"

    # output += "__all__ = [name for name in dir() if name.isupper()]\n"
    xnames = pformat(fnames, width=75, compact=True, indent=4)
    xnames = xnames.replace("[", " ").replace("]", "")
    output += f"__all__ = [\n{xnames}\n]\n"

    return output

output = make_expressions()

def new_module(name: str, code: str = None):
    spec = importlib.util.spec_from_loader(name, None)
    module = importlib.util.module_from_spec(spec)
    if code:
        exec(code, module.__dict__)
    return module

expressions = new_module(f"{PACKAGE}.expressions", output)
expressions.__all__

/var/folders/s8/jtb72jcx6739t9rmc8pl26980000gn/T/ipykernel_16241/1413849124.py:33: DeprecationWarning: Module mintalib.expressions is deprecated and will be removed in a future release!
  exec(code, module.__dict__)


['ABS',
 'ADX',
 'ALMA',
 'ATR',
 'AVGPRICE',
 'BBANDS',
 'BBP',
 'BBW',
 'BOP',
 'CCI',
 'CLAG',
 'CMF',
 'CROSSOVER',
 'CROSSUNDER',
 'CURVE',
 'DEMA',
 'DIFF',
 'DMI',
 'EMA',
 'EXP',
 'FLAG',
 'HMA',
 'KAMA',
 'KELTNER',
 'KER',
 'LAG',
 'LOG',
 'LROC',
 'MACD',
 'MACDV',
 'MAD',
 'MAV',
 'MAX',
 'MDI',
 'MFI',
 'MIDPRICE',
 'MIN',
 'NATR',
 'PDI',
 'PPO',
 'PRICE',
 'QSF',
 'RMA',
 'ROC',
 'RSI',
 'RVALUE',
 'SAR',
 'SHIFT',
 'SIGN',
 'SLOPE',
 'SMA',
 'STDEV',
 'STEP',
 'STOCH',
 'STREAK',
 'SUM',
 'TEMA',
 'TRANGE',
 'TSF',
 'TYPPRICE',
 'UPDOWN',
 'WCLPRICE',
 'WMA']

In [11]:
prices.select(expressions.SMA('close', period=5))


sma
f64
null
null
null
null
0.09206
…
230.256
230.704001
231.326001


In [12]:
module = new_module(f"{PACKAGE}.expressions", output)

outfile = PKGDIR / "expressions.py"

print(f"Updating {outfile.name} ...")

outfile.write_text(output)

Updating expressions.py ...


/var/folders/s8/jtb72jcx6739t9rmc8pl26980000gn/T/ipykernel_16241/1413849124.py:33: DeprecationWarning: Module mintalib.expressions is deprecated and will be removed in a future release!
  exec(code, module.__dict__)


5728

In [13]:
args = "*", 10, 20
src, args = args[0], args[1:]
src, args

('*', (10, 20))

In [14]:
prices.select(pl.struct(pl.col(['high', 'low'])))

high
struct[2]
"{0.099373,0.098943}"
"{0.094211,0.093781}"
"{0.087328,0.086898}"
"{0.089479,0.089049}"
"{0.092061,0.09163}"
…
"{237.490005,232.369995}"
"{232.119995,229.839996}"
"{233.850006,230.520004}"


In [15]:
SMA.__doc__

'\n    Simple Moving Average\n    \n    Args:\n        period (int) : time period, required\n    '